In [1]:
from torchvision import datasets, transforms, models
from torch import nn
import torch.nn.functional as F
import torchvision
import torch

from tqdm import tqdm

import pytorch_lightning as pl
from pytorch_lightning import Trainer, LightningModule
import torchvision.transforms as transforms

from PIL import Image
from simclr import SimCLR
from simclr.modules import NT_Xent
from simclr.modules.transformations import TransformsSimCLR
from simclr.modules.sync_batchnorm import convert_model
from simclr.modules import LARS
from simclr.modules.identity import Identity

import random
from typing import Type, Any, Callable, Union, List, Optional
from torch import Tensor

from simclr.modules.transformations import TransformsSimCLR
from PIL import Image, ImageOps, ImageFilter

import resnet

import os
import argparse
import sys

In [2]:
team_id = 15
team_name = "loSSLess"
email_address = "vvb238@nyu.edu"

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, split, transform, limit=0):
        r"""
        Args:
            root: Location of the dataset folder, usually it is /dataset
            split: The split you want to used, it should be one of train, val or unlabeled.
            transform: the transform you want to applied to the images.
        """

        self.split = split
        self.transform = transform
        
        self.image_dir = os.path.join(root, split)
        label_path = os.path.join(root, f"{split}_label_tensor.pt")

        if limit == 0:
            self.num_images = len(os.listdir(self.image_dir))
        else:
            self.num_images = limit

        if os.path.exists(label_path):
            self.labels = torch.load(label_path).float()
        else:
            self.labels = -1 * torch.ones(self.num_images, dtype=torch.long)

    def __len__(self):
        return self.num_images

    def __getitem__(self, idx):
        idx = int(idx)
        with open(os.path.join(self.image_dir, f"{idx}.png"), 'rb') as f:
            img = Image.open(f).convert('RGB')

        return self.transform(img), self.labels[idx], torch.tensor(idx).float()

In [4]:
class GaussianBlur(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            sigma = random.random() * 1.9 + 0.1
            return img.filter(ImageFilter.GaussianBlur(sigma))
        else:
            return img


class Solarization(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            return ImageOps.solarize(img)
        else:
            return img

In [5]:
class NYUImageNetDataModule(pl.LightningDataModule):
  
    def train_dataloader(self):
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(96, interpolation=Image.BICUBIC),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4,
                                        saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
            GaussianBlur(p=1.0),
            Solarization(p=0.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        trainset = CustomDataset(root='/dataset', split="train", transform=train_transform)
#         train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
        return trainset
    
    def val_dataloader(self):
        eval_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        evalset = CustomDataset(root='/dataset', split="val", transform=eval_transform)
        eval_loader = torch.utils.data.DataLoader(evalset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
        return eval_loader
    
    def unlabeled_dataloader(self):
        unlabeled_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        self.unlabeledset = CustomDataset(root='/dataset', split="unlabeled", transform=unlabeled_transform)
        unlabeled_loader = torch.utils.data.DataLoader(unlabeledset, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)
        return unlabeled_loader
    
    def ssl_train_dataloader(self, batch_size):
        unlabeled_dataset = CustomDataset(root='/dataset', split='unlabeled', transform=TransformsSimCLR(96))
        unlabeled_dataloader = torch.utils.data.DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        return unlabeled_dataloader
        
    def ssl_val_dataloader(self, batch_size):
        val_dataset = CustomDataset(root='/dataset', split='val', transform=TransformsSimCLR(96))
        val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
        return val_dataloader

In [6]:
dataClass = NYUImageNetDataModule()

In [7]:
class ResNetClassifier(LightningModule):
    def __init__(self):
        super().__init__()
#         self.encoder = resnet.get_custom_resnet18()
#         self.encoder.fc = Identity()
#         self.lastLayer = torch.nn.Linear(512, 800)


        self.backbone = torchvision.models.resnet34(zero_init_residual=True)
#         self.backbone = resnet.get_custom_resnet34()
        self.backbone.fc = nn.Identity()
        self.lastLayer = torch.nn.Sequential(
            torch.nn.Linear(512, 1024),
            torch.nn.ReLU(),
            nn.Dropout(p=0.1),
            torch.nn.Linear(1024, 800),
        )
        for layer in self.lastLayer.modules():
           if isinstance(layer, nn.Linear):
                layer.weight.data.normal_(mean=0.0, std=0.01)
                layer.bias.data.zero_()
        
        self.param_groups = [dict(params=self.lastLayer.parameters(), lr=0.01)]
        self.param_groups.append(dict(params=self.backbone.parameters(), lr=0.0001))

        
        self.criterion=torch.nn.CrossEntropyLoss()
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.lastLayer(x)
        return x
    
    def training_step(self, batch, batch_idx):
        data, label, idx= batch
        classProbs = self.forward(data)
        loss = self.criterion(classProbs, label)
        self.log('train_loss', loss)
        return loss
    
    def _evaluate(self, batch, batch_idx, stage=None):
        x, y, _ = batch
        out = self.forward(x)
        logits = F.log_softmax(out, dim=-1)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=-1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f'{stage}_loss', loss, prog_bar=True)
            self.log(f'{stage}_acc', acc, prog_bar=True)

        return loss, acc
    
    def validation_step(self,batch,batch_idx):
        self._evaluate(batch, batch_idx, 'val')[0]
    
    def configure_optimizers(self):
        optimizer = optim.SGD(self.param_groups, 0, momentum=0.9, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS, verbose=True)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

In [8]:
classifier = ResNetClassifier()
classifier.load_state_dict(torch.load(os.path.join('/scratch/vvb238/barlow-34', '27-classifier.pth')))

<All keys matched successfully>

In [9]:
unlabeled_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

entireUnlabeledDataset = CustomDataset(root='/dataset', split="unlabeled", transform=unlabeled_transform)
toBeRankedIndices = torch.tensor([i for i in range(len(entireUnlabeledDataset))])

In [10]:
classifier = classifier.cuda()
activeLearningLoopCount = 3
skimTopPercentage = 5

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

In [ ]:
for i in range(activeLearningLoopCount):
    print("\n\nRunning loop number", i)
    unlabeledFilteredData = torch.utils.data.Subset(entireUnlabeledDataset, toBeRankedIndices.tolist())
    unlabeledFiteredDataLoader = torch.utils.data.DataLoader(unlabeledFilteredData, batch_size=512, shuffle=True, num_workers=6, pin_memory=False)
    allConfidenceScores, predictedLabels = torch.Tensor(), torch.Tensor()
    actualLabels, allIndices, allImageTensors = torch.Tensor(), torch.tensor([]), torch.Tensor()
    
    classifier.eval()
    print("\tStarting the evaluation process with unlabeled data")
    with torch.no_grad():
        # Going through the left over unlabeled set and collecting the confidence for model predictions
        numOfBatches = len(unlabeledFilteredData) / unlabeledFiteredDataLoader.batch_size
        for idx, batch in tqdm(enumerate(unlabeledFiteredDataLoader), total=int(numOfBatches)):
            images, labels, indices = batch

            images = images.cuda()
#             labels = labels.cuda()

            classScores = classifier(images)
            classLogits = F.softmax(classScores, dim=1)
            
            labelConfidence, predictions = torch.max(classLogits.data, 1)
            
            sortedBatchConfidence, sortedBatchConfidencePos = torch.sort(labelConfidence, descending=True)
            topConfidencePos = sortedBatchConfidencePos[:150]
            
            allConfidenceScores = torch.cat((allConfidenceScores, labelConfidence[topConfidencePos].cpu()))
            predictedLabels = torch.cat((predictedLabels, predictions[topConfidencePos].cpu()))
#             actualLabels = torch.cat((actualLabels, labels.cpu()))
            allIndices = torch.cat((allIndices, indices[topConfidencePos].cpu()))
            allImageTensors = torch.cat((allImageTensors, images[topConfidencePos].cpu()))
            
#             if idx and idx % 100 == 0: print("\t\tFinished batch number", idx)
#             if idx and idx % 500 == 0: break
                
            
#             break
        print("\tGot the predictions of" , len(unlabeledFilteredData), " images")
            
        # Sorting all the predictions based on the confidence scores and the argsort
        sortedConfidence, sortedConfidencePos = torch.sort(allConfidenceScores, descending=True)
        print("\tSorted the predictions based on confidence scores")

        # Calculating how many top predictions to retrain the model on
        limit = int(len(unlabeledFilteredData) * (skimTopPercentage/100))
        topConfidencePos = sortedConfidencePos[:limit+2]
        print("\tGot the top ", limit, "confidence indices")
        

        # Fetching the top confidence's index in original dataset
        topConfidenceIndices = allIndices[topConfidencePos]
        # And removing these indices from toBeRankedIndices
        combined = torch.cat((toBeRankedIndices, topConfidenceIndices))
        uniques, counts = combined.unique(return_counts=True)
        toBeRankedIndices = uniques[counts == 1]
        print("\tRemoved the indices of top ranked from further consideration")
        
        # Fetching the top confidence's images and labels
        topConfidenceImages = allImageTensors[topConfidencePos]
        topConfidenceLabels = predictedLabels[topConfidencePos]
        additionalTopConfidenceDataset = torch.utils.data.TensorDataset(topConfidenceImages, topConfidenceLabels, topConfidenceIndices)
        originalAndExtraDataset = torch.utils.data.ConcatDataset((additionalTopConfidenceDataset, dataClass.train_dataloader()))
        originalAndExtraTopConfidenceDataLoader = torch.utils.data.DataLoader(originalAndExtraDataset, batch_size=32, shuffle=True,num_workers=6, pin_memory=True)
        print("\tCombined the original training set and the new dataset")
        
    classifier.train()
    numOfBatches = len(originalAndExtraDataset) / originalAndExtraTopConfidenceDataLoader.batch_size
    print("\tStarting to train the model")
    for epoch in range(15):
        running_loss = 0.0
        for idx, data in tqdm(enumerate(originalAndExtraTopConfidenceDataLoader), total=int(numOfBatches)):
            inputs, labels, idx = data
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = classifier(inputs)
            loss = criterion(outputs, labels.long())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print("\t\tLoss at epoch", epoch, "is", running_loss/numOfBatches)
        scheduler.step(running_loss)
    



Running loop number 0
	Starting the evaluation process with unlabeled data


  9%|▊         | 87/1000 [01:20<09:04,  1.68it/s] 

In [ ]:
eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

evalset = CustomDataset(root='/dataset', split="val", transform=eval_transform)
evalloader = torch.utils.data.DataLoader(evalset, batch_size=256, shuffle=False, num_workers=6)

In [ ]:
net = classifier

net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in evalloader:
        images, labels, idx = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(f"Team {team_id}: {team_name} Accuracy: {(100 * correct / total):.2f}%")

In [ ]:
state = dict(model=classifier.state_dict(),
                 indices=toBeRankedIndices)
torch.save(state, '/scratch/vvb238/iterativeExperiment/checkpoint.pth')